In [2]:
from multiprocessing import Pool,sharedctypes
from functools import partial
from contextlib import closing
import matplotlib.pyplot as plt
import numpy as np
import healpy as hp
import h5py as h5
import os
import sys
import skymapper as skm
import matplotlib.cm as cm
import matplotlib
import copy
from matplotlib.colors import LinearSegmentedColormap

colors = ['#601A4A', '#EE442F','#63ACBE']




from matplotlib import rc
#rc('text', usetex=True)
rc('font', family='serif')
rc('font', size=11)
fontsize='small'


# Read data

In [ ]:
import yaml
import destest
import treecorr


# basic dict props
destest_dict_ = {
    'output_exists' : True,
    'use_mpi'       : False,
    'source'        : 'hdf5',
    'dg'            : 0.01
    }

# Populates a full destest yaml dict for each catalog selection based on the limited catalog input info provided in the common cats.yaml file
def create_destest_yaml( params, name, cal_type, group, table, select_path ):
    """
    Creates the input dictionary structure from a passed dictionary rather than reading froma yaml file.
    """

    destest_dict = destest_dict_.copy()
    destest_dict['load_cache'] = params['load_cache']
    destest_dict['output'] = params['output']
    destest_dict['name'] = name
    destest_dict['filename'] = params['datafile']
    destest_dict['param_file'] = params['param_file']
    destest_dict['cal_type'] = cal_type
    destest_dict['group'] = group
    destest_dict['table'] = table
    destest_dict['select_path'] = select_path
    destest_dict['e'] = ['e_1','e_2']
    destest_dict['Rg'] = ['R11','R22']
    destest_dict['w'] = 'weight'

    return destest_dict

# Build selector (and calibrator) classes from destest for the catalog.
def load_catalog(pipe_params, name, cal_type, group, table, select_path, inherit=None, return_calibrator=None):
    """
    Loads data access and calibration classes from destest for a given yaml setup file.
    """

    # Input yaml file defining catalog
    params = create_destest_yaml(pipe_params, name, cal_type, group, table, select_path)

    # Load destest source class to manage access to file
    source = destest.H5Source(params)

    # Load destest selector class to manage access to data in a structured way
    if inherit is None:
        sel = destest.Selector(params,source)
    else:
        sel = destest.Selector(params,source,inherit=inherit)

    # Load destest calibrator class to manage calibration of the catalog
    if return_calibrator is not None:
        cal = return_calibrator(params,sel)
        return sel, cal
    else:
        return sel

# Read yaml file that defines all the catalog selections used
params = yaml.load(open('cats.yaml'))
params['param_file'] = 'cats.yaml'

# Source catalog
source_selector, source_calibrator = load_catalog(
    params, 'mcal', 'mcal', params['source_group'], params['source_table'], params['source_path'], return_calibrator=destest.MetaCalib)

# Gold catalog
gold_selector = load_catalog(
    params, 'gold', 'mcal', params['gold_group'], params['gold_table'], params['gold_path'], inherit=source_selector)
# BPZ (or DNF) catalog, depending on paths in cats.yaml file (exchange bpz and dnf)
pz_selector = load_catalog(
    params, 'pz', 'mcal', params['pz_group'], params['pz_table'], params['pz_path'], inherit=source_selector)


R1,c,w = source_calibrator.calibrate('e_1') # Optionally pass an additional mask to use when calculating the selection response. The returned R1 is <Rg_1 + Rs_1>. To get an array of R's, use return_wRg=True to get [Rg_1+Rg_2]/2 for each object or return_wRgS=True to include the selection response. return_full=True returns the non-component-averaged version of the full response.
print(R1,c,w)
R2,c,w = source_calibrator.calibrate('e_2')
print(R2,c,w)


# Load ra,dec from gold catalog
ra  = gold_selector.get_col('ra')[0]
dec = gold_selector.get_col('dec')[0]

# Get e1,e2 
g1=source_selector.get_col('e_1')[0]
g2=source_selector.get_col('e_2')[0]
snr =source_selector.get_col('snr')[0]
size_ratio =source_selector.get_col('size_ratio')[0]
w = source_calibrator.calibrate('e_1',weight_only=True) # Optionally pass an additional mask to use when calculating the selection response. The returned R1 is <Rg_1 + Rs_1>. To get an array of R's, use return_wRg=True to get [Rg_1+Rg_2]/2 for each object or return_wRgS=True to include the selection response. return_full=True returns the non-component-averaged version of the full response.
R_array = source_calibrator.calibrate('e_1',return_wRgS=True)

g1 =(g1 - np.mean(g1*w)/np.mean(w))/R1
g2 =(g2 - np.mean(g2*w)/np.mean(w))/R2


/global/homes/m/mgatti/.conda/envs/py3s/lib/python3.6/site-packages/ipykernel_launcher.py:62: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


['T', 'T_err', 'covmat_0_1', 'covmat_1_1', 'covmat_2_2', 'e_1', 'e_2', 'flux_err_i', 'flux_err_r', 'flux_err_z', 'flux_i', 'flux_r', 'flux_z', 'size_ratio', 'snr', 'weight']
using select_path for mask
destest /global/cscratch1/sd/troxel/cats_des_y3/Y3_mastercat_03_31_20.h5 index/select 399263026 100208944
end mask [     7390      7391      7393 ... 399243228 399243235 399243240] [ True  True  True ...  True  True  True]
R11 not in sheared cols
R11 not in sheared cols
R11 not in sheared cols
R11 not in sheared cols
R22 not in sheared cols
R22 not in sheared cols
R22 not in sheared cols
R22 not in sheared cols
skipping sheared columns for e_1
skipping sheared columns for e_1
skipping sheared columns for e_1
skipping sheared columns for e_1
skipping sheared columns for e_2
skipping sheared columns for e_2
skipping sheared columns for e_2
skipping sheared columns for e_2
['bhat', 'cell_wide']
----- e_1 ['e_1', 'e_2']
Rs e_1 0.6735277671413801 0.009213749178185052
0.7178247461393505 0.0 [32

<module 'numpy' from '/global/homes/m/mgatti/.conda/envs/py3s/lib/python3.6/site-packages/numpy/__init__.py'>

# create maps

In [21]:
desy3_map = np.zeros(hp.nside2npix(nside))
desy3_map_w = np.zeros(hp.nside2npix(nside))
desy3_map_w2 = np.zeros(hp.nside2npix(nside))
w_map = np.zeros(hp.nside2npix(nside))
snr_map = np.zeros(hp.nside2npix(nside))
snr_map_w = np.zeros(hp.nside2npix(nside))
SR_map = np.zeros(hp.nside2npix(nside))
SR_map_w = np.zeros(hp.nside2npix(nside))
response_map_w = np.zeros(hp.nside2npix(nside))

sum_we2_2 = np.zeros(hp.nside2npix(nside))
sum_we2_1 = np.zeros(hp.nside2npix(nside))
sum_w2 = np.zeros(hp.nside2npix(nside))
sum_w = np.zeros(hp.nside2npix(nside))
e1_map = np.zeros(hp.nside2npix(nside))
e1_map_w = np.zeros(hp.nside2npix(nside))
e2_map = np.zeros(hp.nside2npix(nside))
e2_map_w = np.zeros(hp.nside2npix(nside))


pix1 = convert_to_pix_coord(ra,dec, nside=nside)
unique_pix1, idx1, idx_rep1 = np.unique(pix1, return_index=True, return_inverse=True)


desy3_map[unique_pix1] += np.bincount(idx_rep1, weights=np.ones(len(pix1)))
desy3_map_w[unique_pix1] += np.bincount(idx_rep1, weights=w)
desy3_map_w2[unique_pix1] += np.bincount(idx_rep1, weights=w**2)
w_map[unique_pix1] += np.bincount(idx_rep1, weights=w)
snr_map_w[unique_pix1] += np.bincount(idx_rep1, weights=snr*w)
snr_map[unique_pix1] += np.bincount(idx_rep1, weights=snr)
SR_map[unique_pix1] += np.bincount(idx_rep1, weights=size_ratio)
SR_map_w [unique_pix1] += np.bincount(idx_rep1, weights=(size_ratio*w))
sum_we2_2[unique_pix1] += np.bincount(idx_rep1, weights=(w*g2)**2)
sum_we2_1[unique_pix1] += np.bincount(idx_rep1, weights=(w*g1)**2)
sum_w2[unique_pix1] += np.bincount(idx_rep1, weights=(w)**2)
sum_w[unique_pix1] += np.bincount(idx_rep1, weights=(w))
response_map_w[unique_pix1] += np.bincount(idx_rep1, weights=R_array)

e1_map_w[unique_pix1] += np.bincount(idx_rep1, weights=g1*w)
e2_map_w[unique_pix1] += np.bincount(idx_rep1, weights=g2*w)
e1_map[unique_pix1] += np.bincount(idx_rep1, weights=g1)
e2_map[unique_pix1] += np.bincount(idx_rep1, weights=g2)


mas_desy3 = desy3_map!=0.
n_eff = desy3_map/(hp.pixelfunc.nside2pixarea(nside,degrees=True)*(60*60)) #/R_ave
n_eff_w = desy3_map_w**2/desy3_map_w2/(hp.pixelfunc.nside2pixarea(nside,degrees=True)*(60*60)) #/R_ave
w_map[mas_desy3] = w_map[mas_desy3]/desy3_map[mas_desy3]
snr_map[mas_desy3] = snr_map[mas_desy3]/desy3_map[mas_desy3]
snr_map_w[mas_desy3] = snr_map_w[mas_desy3]/desy3_map_w[mas_desy3]
SR_map[mas_desy3] = SR_map[mas_desy3]/desy3_map[mas_desy3]
SR_map_w[mas_desy3] = SR_map_w[mas_desy3]/desy3_map_w[mas_desy3]
response_map_w[mas_desy3]=response_map_w[mas_desy3]/desy3_map_w[mas_desy3]
sig_e = np.sqrt((sum_we2_1/sum_w**2+sum_we2_2/sum_w**2)*(sum_w**2/sum_w2)/2.)
e1_map_w[mas_desy3]=e1_map_w[mas_desy3]/desy3_map_w[mas_desy3]
e2_map_w[mas_desy3]=e2_map_w[mas_desy3]/desy3_map_w[mas_desy3]
e1_map[mas_desy3]  =e1_map[mas_desy3]/desy3_map[mas_desy3]
e2_map[mas_desy3]  =e2_map[mas_desy3]/desy3_map[mas_desy3]



dict_maps = dict()
dict_maps['e2_map']=e2_map
dict_maps['e1_map']=e1_map
dict_maps['e1_map_w']=e1_map_w
dict_maps['e2_map_w']=e2_map_w
dict_maps['sig_e']=sig_e
dict_maps['response_map_w']=response_map_w
dict_maps['SR_map_w']=SR_map_w
dict_maps['SR_map']=SR_map
dict_maps['w_map']=w_map
dict_maps['snr_map_w']=snr_map_w
dict_maps['snr_map']=snr_map
dict_maps['n_eff_w']=n_eff_w
dict_maps['n_eff']=n_eff
dict_maps['mas_desy3']=mas_desy3
import pickle
def save_obj( name,obj ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

save_obj('dict_maps',dict_maps)


/global/homes/m/mgatti/.conda/envs/py3s/lib/python3.6/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in true_divide
/global/homes/m/mgatti/.conda/envs/py3s/lib/python3.6/site-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in true_divide


NameError: name 'save_obj' is not defined

In [ ]:

import os
os.environ["PROJ_LIB"] = "C:\\Utilities\\Python\\Anaconda\\Library\\share"
import pylab as plt
import skymap
from skymap import Skymap,McBrydeSkymap,OrthoSkymap

import healpy as hp
import fitsio
import numpy as np
import skymap
from skymap.survey import DESSkymap
import sys
import pickle



nside=1024


import pickle
colors = ['#601A4A', '#EE442F','#63ACBE']

from matplotlib.colors import LinearSegmentedColormap


def load_obj(name):
    
        try:
            with open(name + '.pkl', 'rb') as f:
                return pickle.load(f)#, encoding='latin1')
        except:
            with open(name + '.pkl', 'rb') as f:
                return pickle.load(f, encoding='latin1')
            
            
def make_plot(mapp,pix_mask,savedir,title,xsize=5000,smooth=10.0/60,smoothing=True,vmin=-0.01,vmax=0.01):
    fig = plt.figure(figsize=(9,6))


    smap = skymap.DESSkymap()
    ax=plt.gca()
    

    
    plt.sca(ax)
    if smoothing:
        smap.draw_hpxmap(mapp[pix_mask],pix_mask,nside,cmap=cmap,vmin=vmin,vmax=vmax,xsize=xsize,smooth=smooth) #
    else:
        smap.draw_hpxmap(mapp[pix_mask],pix_mask,nside,cmap=cmap,vmin=vmin,vmax=vmax,xsize=xsize) #
    smap.draw_inset_colorbar(ticks=[vmin,vmax],format='%.2f',fontsize=10,bbox_to_anchor=(-0.05,-0.0,1,1))
    
    plt.title(title,y=1.08,fontsize=18)
    plt.savefig(savedir,bbox_inches='tight')
    plt.show()

des_c1_cmap = LinearSegmentedColormap.from_list('mycmap', list(zip(np.linspace(0,1,2), ['#ffffff', colors[0]])))

cmap = des_c1_cmap
pixels = np.arange(hp.nside2npix(nside))
pix_mask =pixels[dictt['mas_desy3']]



make_plot((dictt['n_eff_w']),pix_mask,title = 'n_eff (H12, with weights)',vmin = 3., vmax = 8.,savedir='./figures_shearcat/neffw.pdf')
make_plot((dictt['sig_e']),pix_mask,title ='sigma_e (with weights)',vmin = 0.25, vmax = 0.3,savedir='./figures_shearcat/sige.pdf')
make_plot((dictt['response_map_w']),pix_mask,title ='Response (with weights)',vmin = 0.65, vmax = 0.75,savedir='./figures_shearcat/responsew.pdf')
make_plot(np.log10(dictt['snr_map']+1),pix_mask,title ='log10 snr (no weights)',vmin = 1., vmax = 2,savedir='./figures_shearcat/snr.pdf')
make_plot((dictt['SR_map']),pix_mask,title ='size ratio (no weights)',vmin = 0.5, vmax = 2.5,savedir='./figures_shearcat/SR.pdf')


